# 자동 미분

In [1]:
import tensorflow as tf
import numpy as np

텐서플로의 자동 미분

tf.GridientTape

tf.GridientTape는 컨텍스트(context) 안에서 실행된 모든 연산을 테이프(tape)에 기록
그 다음 텐서플로는 후진 방식 자동 미분(reverse mode differentiation)을 사용해 테이프에 기록된 연산의 그래디언트를 계산

- Scalar 를 Scalar로 미분

In [2]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x**2

In [3]:
y

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [4]:
# dy = 2x * dx
dy_dx = tape.gradient(y,x)
dy_dx.numpy()

6.0

- Scalar 를 Vector로 미분

In [5]:
w = tf.Variable(tf.random.normal((3, 2)), name="w")
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
    y = x @ w + b
    loss = tf.reduce_mean(y**2)

In [6]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

[dl_dw, dl_db]

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ -5.5325084,   2.8777187],
        [-11.065017 ,   5.7554374],
        [-16.597525 ,   8.633156 ]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-5.5325084,  2.8777187], dtype=float32)>]

In [7]:
my_vars = {
    'w' : w,
    'b' : b
}

grad = tape.gradient(loss, my_vars)
grad['b']

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-5.5325084,  2.8777187], dtype=float32)>

- 자동 미분 컨트롤하기!
    - tf.Variable만 기록
    - A variable + tensor는 tensor를 반환
    - trainable 조건으로 미분 기록을 제어

In [9]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')

# Not trainable
x1 = tf.Variable(3.0, name='x1', trainable=False)

# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0

# Not a variable
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
    y = (x0**2) + (x1**2) + (x2**2)
    
grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
    print(g)

tf.Tensor(6.0, shape=(), dtype=float32)
None
None
None


#### 기록되고 있는 variable 확인하기

In [10]:
tape.watched_variables()

(<tf.Variable 'x0:0' shape=() dtype=float32, numpy=3.0>,)